<a href="https://colab.research.google.com/github/cowiety/energy-forecast/blob/main/Data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenWeather API Script


Import statements

In [ ]:
import requests
from tkinter import *
import math
import time
import csv
import pandas as pd

Global variable declaration

In [ ]:
cities = {
                  "Toronto": [43.7001, -79.4163, 5429524], 
                  "Ottawa": [45.4112, -75.6981, 989567], 
                  "Hamilton": [43.2334, -79.9496, 693645], 
                  "Kitchener": [43.4254, -80.5112, 470015], 
                  "London": [42.9834, -81.233, 383437], 
                  "Oshawa": [43.9001, -78.8496, 308875], 
                  "Windsor": [42.3001, -83.0165, 287069], 
                  "St. Catharines": [43.1668, -79.2496, 229246], 
                  "Barrie": [44.4001, -79.6663, 145614], 
                  "Guelph": [43.5501, -80.2497, 132397], 
                  "Kingston": [44.2298, -76.481, 117660]
}
total_pop = 9187049
api_key = "ffe15c3fc982bc7ae70c6edfa2fc779c"

Get data for time (hour), cloudiness, temperature, humidity and wind speed for each hour of the day

In [ ]:
def get_daily_weather(city):
  # format city dictionary into API parameters
  lat = cities[city][0]
  lon = cities[city][1]
  url = f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude=current,minutely,daily,alerts&appid={api_key}"

  # pulls API request and converts to json
  response = requests.get(url).json()

  # initialize 4 empty lists
  hours, clouds, temp, humidity, w_speed = [], [], [], [], []
  # looping through next 24 hours of weather data
  for i in range(0,24):
    hour = time.gmtime(int(response['hourly'][i]['dt'] + response['timezone_offset']))[3]
    hours.append(hour)
    cloudiness = response['hourly'][i]['clouds']
    clouds.append(cloudiness)
    temperature = "{:.2f}".format(response['hourly'][i]['temp'] - 273.15)
    temp.append(float(temperature))
    humidex = response['hourly'][i]['humidity']
    humidity.append(humidex)
    wind = response['hourly'][i]['wind_speed']
    w_speed.append(wind)
  return hours, clouds, temp, humidity, w_speed

Collect the data for each city and split the data into 4 arrays by feature

In [ ]:
# Create file or overwrite if exists
open('results.csv', 'w+')
weighted_clouds, weighted_temp, weighted_humidity, weighted_w_speed = [], [], [], []
for city in cities:
  hours, clouds, temp, humidity, w_speed = get_daily_weather(city)

  # weight each city's data point according to the population of that city
  clouds = [element * cities[city][2] for element in clouds]
  temp = [element * cities[city][2] for element in temp]
  humidity = [element * cities[city][2] for element in humidity]
  w_speed = [element * cities[city][2] for element in w_speed]

  # concatenate the data from each city into individual arrays for data processing later
  weighted_clouds = weighted_clouds + clouds 
  weighted_temp = weighted_temp + temp
  weighted_humidity = weighted_humidity + humidity  
  weighted_w_speed = weighted_w_speed + w_speed

Calculate the weighted average for each hour of the day and export to csv

In [ ]:
# initialize 4 empty lists
ave_clouds, ave_temp, ave_humidity, ave_w_speed = [0]*24, [0]*24, [0]*24, [0]*24

# loop through the concatenated array for each hour of the day
for i in range(len(hours)):
  # loop through the same concatenated array for each city 
  for j in range(len(cities)):
    # since there are 24 hours x 11 cities in each list
    # [j*24 + i - 1] collects all data for the ith hour
    ave_clouds[i] = ave_clouds[i] + weighted_clouds[j*24 + i - 1]
    ave_temp[i] = ave_temp[i] + weighted_temp[j*24 + i - 1]
    ave_humidity[i] = ave_humidity[i] + weighted_humidity[j*24 + i - 1]
    ave_w_speed[i] = ave_w_speed[i] + weighted_w_speed[j*24 + i - 1]

# format the temperature and wind speed to 3 decimal places
formatter = "{0:.3f}"
# format the data appropriately and divide each element by the total population
ave_clouds = [round(element / total_pop) for element in ave_clouds]
ave_temp = [formatter.format(element / total_pop) for element in ave_temp]
ave_humidity = [round(element / total_pop) for element in ave_humidity]
ave_w_speed = [formatter.format(element / total_pop) for element in ave_w_speed]

# format all the data into a new DataFrame
data = {'Hour': hours,
        'Cloudiness (%)': ave_clouds,
        'Temp (°C)': ave_temp,
        'Humidity (%)': ave_humidity,
        'Wind Speed (m/s)': ave_w_speed
}
df = pd.DataFrame(data)

# convert DataFrame to csv file and print to terminal
df.to_csv('results.csv', index=False)
print(df)

    Hour  Cloudiness (%) Temp (°C)  Humidity (%) Wind Speed (m/s)
0     17              87    -0.697            71            8.252
1     18              35    -4.110            64            4.691
2     19              27    -4.189            65            4.374
3     20              35    -3.932            65            4.417
4     21              49    -3.521            64            4.630
5     22              64    -2.834            63            5.207
6     23              79    -2.040            63            5.895
7      0              94    -1.221            68            6.293
8      1              95    -1.002            76            6.526
9      2              96    -0.554            80            6.770
10     3             100    -0.071            84            6.731
11     4             100     0.304            87            6.503
12     5             100     0.758            91            6.849
13     6             100     1.359            93            7.133
14     7  

# Creating Holiday CSV File

import statements

In [ ]:
import pandas as pd #data processing, CSV I/O
import holidays
from datetime import date

load holiday library into object variable, import calendar.csv into dataframe

In [ ]:
can_holidays = holidays.Canada()
df = pd.read_csv('calendar.csv')
df.head(10)

FileNotFoundError: ignored

add a column to dataframe called 'isHoliday'

In [ ]:
df['isHoliday'] = range(len(df))

check if each date is a holiday and place the result in the isHoliday column

In [ ]:
for row in range(len(df)-1):

  df['isHoliday'][row] = date(df['year'][row], df['month'][row], df['day'][row]) in can_holidays

df.head(10)

export dataframe to csv

In [ ]:
df.to_csv('holidays.csv', index=False)

# Cleaning IESO Demand Data

Produces an up-to-date csv file of Ontario hourly demand data from the previous year

In [ ]:
import pandas as pd #data processing, CSV I/O
import datetime 

Downloads CSVs from IESO website for the current and prior year

In [ ]:
current_year = date.today().year
last_year = current_year - 1
last_year_link = 'http://reports.ieso.ca/public/Demand/PUB_Demand_{}.csv'.format(last_year)
current_year_link = 'http://reports.ieso.ca/public/Demand/PUB_Demand_{}.csv'.format(current_year)
df_1 = pd.read_csv(last_year_link, skiprows=3)
df_2 = pd.read_csv(current_year_link, skiprows=3)
df_2.head(10)

,Date,Hour,Market Demand,Ontario Demand
0,2022-01-01,1,15150,13543
1,2022-01-01,2,15344,13254
2,2022-01-01,3,14686,12683
3,2022-01-01,4,14400,12491
4,2022-01-01,5,14043,12493
5,2022-01-01,6,14585,12586
6,2022-01-01,7,14688,12772
7,2022-01-01,8,15113,12915
8,2022-01-01,9,15010,13265
9,2022-01-01,10,15644,13861


Function splits the date column into year, month and day

In [ ]:
def reformat_df(df):
  df['Year'] = df.Date.apply(lambda x: int(x[:4]))
  df['Month'] = df.Date.apply(lambda x: int(x[5:7]))
  df['Day'] = df.Date.apply(lambda x: int(x[8:]))
  df['Demand'] = df['Market Demand']
  df.drop(['Ontario Demand', 'Market Demand', 'Date'], axis=1, inplace = True)

Apply the previous function and remove rows that are earlier than one year ago

In [ ]:
reformat_df(df_1)
reformat_df(df_2)

first_index = df_1[(df_1.Day == date.today().day) & (df_1.Hour == (datetime.datetime.now().hour - 5)) & (df_1.Month == date.today().month)].index[0]
df_1.drop(list(range(first_index)), inplace = True)
full_year = df_1.append(df_2, ignore_index = True)
full_year.head(10)

,Hour,Year,Month,Day,Demand
0,18,2021,1,8,21407
1,19,2021,1,8,21434
2,20,2021,1,8,20661
3,21,2021,1,8,20202
4,22,2021,1,8,19243
5,23,2021,1,8,18362
6,24,2021,1,8,17749
7,1,2021,1,9,17115
8,2,2021,1,9,16622
9,3,2021,1,9,16346


Reformat the date columns into ISO formatting 

In [ ]:
full_year['Date'] = full_year.apply(lambda row: '{:04d}-{:02d}-{:02d}'.format(row['Year'], row['Month'], row['Day']), axis = 1)
full_year.drop(['Day', 'Year', 'Month'], inplace = True, axis = 1)
full_year.head(10)

,Hour,Demand,Date
0,18,21407,2021-01-08
1,19,21434,2021-01-08
2,20,20661,2021-01-08
3,21,20202,2021-01-08
4,22,19243,2021-01-08
5,23,18362,2021-01-08
6,24,17749,2021-01-08
7,1,17115,2021-01-09
8,2,16622,2021-01-09
9,3,16346,2021-01-09


Export the dataframe into a csv file named with the current date

In [ ]:
full_year.to_csv('Demand Data ({:04d}-{:02d}-{:02d}).csv'.format(date.today().year, date.today().month, date.today().day), index = False)